# Instructions

# MetAltas targeted analysis workflow - streamlined

NOTES:
+ right now the package import statements and wrapper functions are in one big file (*metatlas_targeted_wrapper_functions.py*) that gets imported - not sure what is best practice in python
+ RT adjuster nb has: `import matplotlib.pyplot as plt` while main nb has `import matplotlib.pyplot as plot`  --> i changed everything here to `plot`
+ For simplicty, for now I took out the code to upload an atlas from a spreadsheet - all the code here uses the list of atlases that are already in the database supplied by Katherine.
+ i made a new Class for RT_model which holds a name, coef, and intercept. create_rt_adjustment_model passes back two instances of this class - one for linear, one for poly - and the user choses which one to pass to apply_rt_adjustment
+ this is set up to calculate and pickle a new hits variable once for each atlas version - not sure if that's correct?
+ i added two small functions to help with adjusting RT bars
    + `find_isomers` finds any other compounds in the atlas with the same mz
    + `get_compound_notes` takes a csv file as input with the following columns: 'label' 'polarity' 'note' (at least, any order is ok) and prints any rows where label matches the label of a compound of interest in the current atlas 
+ the `export_results` function checks to see whether plot directories are empty before adding new plots to them
    + there is a parameter `remove_existing_plots` which is set to False by default
    + if False, a warning will be printed when a plot directory isn't empty and no new plots will be generated
    + if True, the existing directory will be removed before new plots are generated
+ things to do:
    + make a function that restarts the kernel
    + implement checkpoint files so its clear what's already been run

### Parameters
Run these two blocks after every kernel reset

In [ ]:
### user-specific parameters
scripts_dir='/global/homes/g/greensi/scripts/metatlas' # <- EDIT THIS directory where metatlas repo code is
project_directory='/global/homes/g/greensi/metatlasData'    # <- EDIT THIS directory to store all output data
my_id='%SIGv1%' # <- EDIT THIS personal identifier
username='greensi' # <- EDIT THIS

import sys, os
sys.path.insert(0,scripts_dir)
    

In [1]:
### analysis-specific parameters
project_name='20210115_JGI-AK_SR-KP_506299_Toblongifolia_final_QE-HF_HILICZ_USHXG01531' # <- EDIT THIS

### Import Packages and Functions 
run this after every kernel restart

In [ ]:
%matplotlib notebook
from metatlas_targeted_wrapper_functions import * # <- the file metatlas_targeted_wrapper_functions.py must be on your path

# CHECK FOR SUCCESS
if(fa.__file__ != scripts_dir+"/metatlas/tools/fastanalysis.py"):
    print("error: fa.__file__ is "+fa.__file__)
else:
    print("setup successful")


### Step 1: RT-adjust atlases
+ Block 1 will print a list of options - pick the desired index and fill in `selected_column=` in Block 2
+ Block 2 returns two variables for the two types of models - fill in `model=` with the name of one of them in Block 3
+ Restart after block 3

In [ ]:
## extra packages for rt-adjustment
%matplotlib inline
%env HDF5_USE_FILE_LOCKING=FALSE
from __future__ import division
import itertools
import math
from matplotlib import gridspec
import matplotlib.ticker as mticker
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error as mae

analysis_round = "data_qc"
output_dir=setup_notebook(project_directory,project_name,analysis_round)

In [ ]:
# block 1 - only run this once!! (it creates groups for the project)
rts_df, atlas_df, myAtlas = setup_rt_adjustment(project_name,my_id,output_dir)  

In [ ]:
# block 2
selected_column = #[0-9]
model_linear, model_poly = create_rt_adjustment_model(selected_column,rts_df,atlas_df)

In [ ]:
# block 3
model = #model_linear or model_poly
apply_rt_adjustment(model,project_name,my_id,output_dir)

In [ ]:
## RESTART

### Step 2: Adjust RT bounds

#### 2a. choose setup
+ run one of the following blocks to select or change which run mode you're working on
+ your selection will be saved to a text file and read in automatically every time you run the blocks in the following sections

In [ ]:
analysis_round='ISTD' 
polarity="POS"
write_run_options(project_directory,project_name,analysis_round,polarity)

In [ ]:
analysis_round='ISTD'  
polarity="NEG"
write_run_options(project_directory,project_name,analysis_round,polarity)

In [ ]:
analysis_round='FINAL' 
polarity="POS"
write_run_options(project_directory,project_name,analysis_round,polarity)

In [ ]:
analysis_round='FINAL'
polarity="NEG"
write_run_options(project_directory,project_name,analysis_round,polarity)

#### 2b. choose an atlas
+ run these two blocks every time you restart kernel or select a new setup above

In [ ]:
# block 1
analysis_round, polarity = read_run_options(project_directory,project_name)
output_dir=setup_notebook(project_directory,project_name,analysis_round)
atlases_options=pick_atlas(project_name,my_id,analysis_round,polarity,username)

In [ ]:
# block 2
selected_atlas= 1 # enter an index from the list produced by the code block above
final_round=False # set to true when you are ready to export

my_atlas=atlases_options[selected_atlas]
metatlas_dataset = make_metatlas_dataset(my_atlas,final_round,project_name,my_id,output_dir,polarity)

#### 2c. filter atlas 
make sure you've selected the appropriate atlas by running the two blocks from the previous section

In [ ]:
if analysis_round == "FINAL":  ## dont run this block for ISTD
    filter_atlas(my_atlas,metatlas_dataset,project_name,my_id,output_dir,polarity,num_data_points_passing = 5,peak_height_passing = 4e5)

# RESTART KERNEL

#### 2d. adjust rt bars

In [ ]:
hits = get_msms_hits(metatlas_dataset,my_atlas,output_dir) ## will load pickled hits if available

In [ ]:
import warnings; warnings.simplefilter('ignore')
a = dp.adjust_rt_for_selected_compound(metatlas_dataset, msms_hits=hits, 
                                       peak_flags="", msms_flags="", 
                                       color_me = [['red','ISTD'],['yellow','InjBL'],['blue','Ctrl']], 
                                       compound_idx=0,alpha=0.5,width=16,height=3) #,y_max = 1e6

In [ ]:
## use this function to find any other compounds in the atlas with the same mz as a specified compound
find_isomers(13,my_atlas)

In [ ]:
## use this function to pull up any notes you've stored for a given compound 
## notes_file should be a csv file with the following columns : 'label' 'polarity' 'note' (any order is ok, can have more columns too)
## the 'label' field is used to match compounds in the csv with compounds in the current atlas

notes_file=os.path.join(project_directory,"compound_identification_notes.csv")
get_compound_notes(4,polarity,my_atlas,notes_file)

In [ ]:
# RESTART KERNEL TO APPLY CHANGES

#### remove marked compounds

In [ ]:
##  kept_string (default: 'kept') will be appended to the name of the atlas selected in 2b and stored as a new atlas
##  change kept_string to something else (ie kept_2, kept_3) if you re-select the original (filtered) atlas and want to make a new version with different compounds removed
remove_marked_compounds(metatlas_dataset,my_atlas,polarity,kept_string="kept") 

In [ ]:
# RESTART KERNEL TO APPLY CHANGES

#### export data

In [ ]:
export_results(metatlas_dataset,hits,my_atlas,output_dir,polarity,remove_existing_plots=False)